In [2]:
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from faker import Faker

In [3]:
client = QdrantClient(host="localhost", port=6333)
client

In [5]:
my_collection = "first_collection"
client.create_collection(
    collection_name=my_collection,
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

True

In [7]:
data= np.random.uniform(low= -1.0, high=1.0, size=(1_000, 100))
index = list(range(1_000))

In [8]:
data[:2, :10]

array([[-0.03014071, -0.74019734, -0.20964797, -0.62847091,  0.69803559,
         0.26107119,  0.44284354, -0.69906928,  0.99813325,  0.52152067],
       [-0.67595696, -0.40824292,  0.60325621, -0.57569899,  0.12777019,
         0.39316794,  0.36555069,  0.84878534,  0.44136644,  0.33506853]])

In [11]:
index[-10:]

[990, 991, 992, 993, 994, 995, 996, 997, 998, 999]

In [13]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [18]:
client.retrieve(
    collection_name=my_collection,
    ids= [10,14,500],
    #with_vectors=True
)

[Record(id=10, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=14, payload={}, vector=None, shard_key=None, order_value=None),
 Record(id=500, payload={}, vector=None, shard_key=None, order_value=None)]

In [19]:
fake_something = Faker()
fake_something.name(), fake_something.address()

('Christopher Moran', '452 Ronald Lodge Apt. 966\nAlfredton, MN 13547')

In [20]:
payload = []

for i in range(1_000):
    payload.append(
        {
            "artist": fake_something.name(),
            "Song": " ".join(fake_something.word()),
            "URL": fake_something.url(),
            "year": fake_something.year(),
            "Country": fake_something.country(),
        }
    )

payload[:3]

[{'artist': 'Jay Bishop',
  'Song': 'b l u e',
  'URL': 'http://www.hernandez.com/',
  'year': '2011',
  'Country': 'Eritrea'},
 {'artist': 'Ashlee Bryan',
  'Song': 'a u t h o r i t y',
  'URL': 'https://evans.com/',
  'year': '1998',
  'Country': 'Mali'},
 {'artist': 'Jake Conley',
  'Song': 's t a n d a r d',
  'URL': 'http://www.cruz.com/',
  'year': '1981',
  'Country': 'Russian Federation'}]

In [22]:
client.upsert(
    collection_name=my_collection,
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

semantic search

In [26]:
living_la_vida_loca= np.random.uniform(low= -1.0, high=1.0, size=(100)).tolist()
living_la_vida_loca[:5]

[-0.9244923548536106,
 0.41494791205048265,
 -0.7014746410983383,
 0.6600260565992424,
 0.3820291759481309]

In [27]:
client.search(
    collection_name=my_collection,
    query_vector=living_la_vida_loca,
    limit=10
)

[ScoredPoint(id=907, version=1, score=0.39742094, payload={'artist': 'Kelli Miller', 'Song': 's o m e o n e', 'URL': 'https://www.moody.com/', 'year': '1993', 'Country': 'Malawi'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=747, version=1, score=0.30809447, payload={'artist': 'Howard Edwards', 'Song': 'g r e e n', 'URL': 'https://maynard.org/', 'year': '2018', 'Country': 'China'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=295, version=1, score=0.27436155, payload={'artist': 'Amanda Rich', 'Song': 's u f f e r', 'URL': 'https://www.sandoval.com/', 'year': '1975', 'Country': 'Venezuela'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=500, version=1, score=0.2534599, payload={'artist': 'Brandi Hall', 'Song': 'e f f e c t', 'URL': 'https://scott-smith.com/', 'year': '2022', 'Country': 'Ecuador'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=947, version=1, score=0.24528192, payload={'artist': 'Anthony Parrish',

In [45]:
aussie_songs = models.Filter(
    must = [
        models.FieldCondition(
            key="country",match=models.MatchValue(value="Australia")
        )
    ]
)
aussie_songs

Filter(should=None, min_should=None, must=[FieldCondition(key='country', match=MatchValue(value='Australia'), range=None, geo_bounding_box=None, geo_radius=None, geo_polygon=None, values_count=None)], must_not=None)

In [57]:
client.query_points(
    collection_name=my_collection,
    query=living_la_vida_loca,
    query_filter=aussie_songs,
    with_payload=models.PayloadSelectorExclude(exclude=["year"]),
    limit=5
).points

[]

In [59]:
client.recommend(
    collection_name=my_collection,
    positive=[17],
    limit=5
)

[ScoredPoint(id=684, version=1, score=0.35292754, payload={'artist': 'Christopher Hicks', 'Song': 'm a y b e', 'URL': 'http://www.lynch.com/', 'year': '2015', 'Country': 'Libyan Arab Jamahiriya'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=707, version=1, score=0.30003223, payload={'artist': 'Nicholas Martinez', 'Song': 'l a w y e r', 'URL': 'http://www.daugherty.com/', 'year': '2024', 'Country': 'Indonesia'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=514, version=1, score=0.28852525, payload={'artist': 'Jonathan Lamb', 'Song': 'f e w', 'URL': 'http://sanders.com/', 'year': '1993', 'Country': 'Barbados'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=660, version=1, score=0.28100246, payload={'artist': 'Ariel Alexander', 'Song': 'a c t', 'URL': 'http://www.johnson-watson.com/', 'year': '1997', 'Country': 'Gambia'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=77, version=1, score=0.2789402, payload={'artist

In [61]:
client.query_points(
    collection_name=my_collection,
    query=models.RecommendQuery(recommend=models.RecommendInput(
        positive=[17],
        negative=[120, 180]
    )),
    query_filter=models.Filter(
        must=[models.FieldCondition(key="country", match=models.MatchValue(value="Dominican Republic"))]
    ),
    limit=5
).points

[]